In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from IPython import get_ipython
import warnings
warnings.filterwarnings("ignore")


In [ ]:
data = pd.read_csv('adults.txt')

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data.shape

In [ ]:
data.columns

In [ ]:
data.duplicated().sum()

In [ ]:
data = data.drop_duplicates()

In [ ]:
data.isnull().sum()

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.nunique()

In [ ]:
data = data.drop(['fnlwgt', 'educational-num'], axis=1)
col_names = data.columns
for c in col_names: 
    data = data.replace("?", np.NaN)
    
data = data.apply(lambda x: x.fillna(x.value_counts().index[0]))

In [ ]:
from sklearn import preprocessing

In [ ]:
data.replace(['Divorced', 
              'Married-AF-spouse',
              'Married-civ-spouse', 
              'Married-spouse-absent', 
              'Never-married', 'Separated',
              'Widowed'],
             
             ['divorced',
              'married',
              'married',
              'married', 
              'not married',
              'not married', 
              'not married'], 
             
             inplace=True)

category_col = ['workclass', 'race', 'education', 
                'marital-status', 'occupation',
                'relationship', 'gender', 'native-country', 
                'income']

labelEncoder = preprocessing.LabelEncoder()
mapping_dict = {}
for col in category_col: 
    data[col] = labelEncoder.fit_transform(data[col]) 

    le_name_mapping = dict(zip(labelEncoder.classes_, 
                               labelEncoder.transform(labelEncoder.classes_)))   
    
    mapping_dict[col] = le_name_mapping

print(mapping_dict)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [ ]:
X = data.values[:, 0:12]
Y = data.values[:, 12]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X, Y, 
                                                    test_size = 0.3,
                                                    random_state = 100) 

dt_clf_gini = DecisionTreeClassifier(criterion = "gini", 
                                     random_state = 100, 
                                     max_depth = 5, 
                                     min_samples_leaf = 5) 

dt_clf_gini.fit(X_train, y_train)
y_pred_gini = dt_clf_gini.predict(X_test) 
print ("Decision Tree using Gini Index\nAccuracy is ", accuracy_score(y_test, y_pred_gini)*100 )

In [ ]:
from flask import * 
import pickle
app = Flask(__name__)

# prediction function
def ValuePredictor(to_predict_list): 
    to_predict = np.array(to_predict_list).reshape(1, 12) 
    loaded_model = pickle.load(open("model.pkl", "rb")) 
    result = loaded_model.predict(to_predict) 
    return result[0]

@app.route('/result', methods = ['POST'])
def result(): 
    if request.method == 'POST': 
        to_predict_list = request.form.to_dict() 
        to_predict_list = list(to_predict_list.values()) 
        to_predict_list = list(map(int, to_predict_list)) 
        result = ValuePredictor(to_predict_list) 
        
        if int(result)== 1: 
            prediction ='Income more than 50K' 
        else: 
            prediction ='Income less that 50K' 

        return render_template("result.html", prediction = prediction)
if __name__ == '__main__': 
    app.run(debug = True)